<h1><b>Packages</h1>

In [13]:
import pandas as pd
import gc
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

!pip install category-encoders --q
from category_encoders import BinaryEncoder

<h1><b>Importing the Dataset</h1>

In [2]:
!pip install kaggle --q

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c ieee-fraud-detection
!unzip ieee-fraud-detection.zip
!rm -rf ieee-fraud-detection.zip

  0% 0.00/118M [00:00<?, ?B/s]
100% 118M/118M [00:00<00:00, 1.84GB/s]
Archive:  ieee-fraud-detection.zip
  inflating: sample_submission.csv   
  inflating: test_identity.csv       
  inflating: test_transaction.csv    
  inflating: train_identity.csv      
  inflating: train_transaction.csv   


<h1><b>Pre-Processing the Dataset</h1>

In [5]:
train_trans = pd.read_csv('train_transaction.csv')
train_iden = pd.read_csv('train_identity.csv')

In [6]:
train = train_trans.merge(train_iden, on='TransactionID', how='left')

del train_trans
del train_iden
gc.collect()

0

In [5]:
print(train.shape)

(590540, 434)


In [7]:
missing_percent = train.isna().mean()
missing_percent[missing_percent > 0.85]

,0
dist2,0.936284
D6,0.876068
D7,0.934099
D8,0.873123
D9,0.873123
...,...
id_27,0.991247
id_30,0.868654
id_32,0.868619
id_33,0.875895


- **From this we can clearly see that 74 columns have missing values which are greater than 85%**
- **We will be dropping those columns as they would just create noise and won't help in model training**

In [8]:
missing_percentage = train.isna().mean()
cols_to_drop = missing_percentage[missing_percentage > 0.85].index

train = train.drop(columns=cols_to_drop)
len(cols_to_drop)

74

In [9]:
y=train['isFraud']
X=train.drop(columns=['isFraud'])

del train
gc.collect()

0

In [10]:
num_cols = X.select_dtypes(include=np.number).columns
cat_cols = X.select_dtypes(exclude=np.number).columns

num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')

X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

In [11]:
encoder = BinaryEncoder(cols=cat_cols)
X = encoder.fit_transform(X)

In [12]:
import gc

del num_cols
del cat_cols
del num_imputer
del cat_imputer

gc.collect()


6

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y            # keeps fraud ratio same in both sets
)

In [16]:
X.to_parquet("X_processed.parquet")

y_df = pd.DataFrame({'isFraud': y})
y_df.to_parquet("y_processed.parquet")

print("Saved processed data ✅")

Saved processed data ✅
